In [1]:
import re
import matplotlib.pyplot as plt
from collections import defaultdict
from ipywidgets import interact, FloatRangeSlider, FileUpload, VBox, Output, HTML, Button
from datetime import datetime

In [2]:
def parse_log_data(content):
    data = {
        'c': defaultdict(lambda: {'t': [], 'a': [], 'e': [], 'ae': []}),
        's': defaultdict(lambda: {'t': [], 'a': [], 'e': [], 'ae': []})
    }
    pattern = r'(\d{2}:\d{2}:\d{2})\s+\[TRACE\].*?t:([\d.]+) a:([-\d.]+) \((\d+)Hz\)'
    pattern2 = r'(\d{2}:\d{2}:\d{2})\s+\[TRACE\].*?t:([\d.]+) e:([-\d.]+) a:([-\d.]+) ae:([-\d.]+) \((\d+)Hz\)'

    base_time = None
    
    for line in content.split('\n'):
        if 'Composite Generator ticking' in line:
            match = re.search(pattern, line)
            if match:
                timestamp_str = match.group(1)
                phase_t = float(match.group(2))
                a = float(match.group(3))
                freq = match.group(4)
                
                # Parse timestamp (HH:MM:SS)
                time_obj = datetime.strptime(timestamp_str, '%H:%M:%S')
                # Convert to seconds from midnight
                log_time = time_obj.hour * 3600 + time_obj.minute * 60 + time_obj.second
                
                # Set base time on first entry
                if base_time is None:
                    base_time = log_time
                
                # Calculate actual time relative to first log entry
                actual_time = (log_time - base_time) + phase_t
                
                data['c'][freq]['t'].append(phase_t)
                data['c'][freq]['a'].append(a)
                data['c'][freq]['e'].append(0)
                data['c'][freq]['ae'].append(0)
            elif (match := re.search(pattern2, line)):
                timestamp_str = match.group(1)
                phase_t = float(match.group(2))
                e = float(match.group(3))
                a = float(match.group(4))
                ae = float(match.group(5))
                freq = match.group(6)
                
                # Parse timestamp (HH:MM:SS)
                time_obj = datetime.strptime(timestamp_str, '%H:%M:%S')
                # Convert to seconds from midnight
                log_time = time_obj.hour * 3600 + time_obj.minute * 60 + time_obj.second
                
                # Set base time on first entry
                if base_time is None:
                    base_time = log_time
                
                # Calculate actual time relative to first log entry
                actual_time = (log_time - base_time) + phase_t
                
                data['c'][freq]['t'].append(phase_t)
                data['c'][freq]['e'].append(e)
                data['c'][freq]['a'].append(a)
                data['c'][freq]['ae'].append(ae)
        if 'Tone Generator ticking' in line:
            match = re.search(pattern2, line)
            if match:
                timestamp_str = match.group(1)
                phase_t = float(match.group(2))
                e = float(match.group(3))
                a = float(match.group(4))
                ae = float(match.group(5))
                freq = match.group(6)
                
                # Parse timestamp (HH:MM:SS)
                time_obj = datetime.strptime(timestamp_str, '%H:%M:%S')
                # Convert to seconds from midnight
                log_time = time_obj.hour * 3600 + time_obj.minute * 60 + time_obj.second
                
                # Set base time on first entry
                if base_time is None:
                    base_time = log_time
                
                # Calculate actual time relative to first log entry
                actual_time = (log_time - base_time) + phase_t
                
                data['s'][freq]['t'].append(phase_t)
                data['s'][freq]['e'].append(e)
                data['s'][freq]['a'].append(a)
                data['s'][freq]['ae'].append(ae)
    
    return data

In [3]:
def plot_log(file_path, composite=True):
    """
    Plot generator output from a log file with interactive time range slider.
    
    Args:
        file_path: Path to the log file
    """
    with open(file_path, 'r') as f:
        content = f.read()
    
    data = parse_log_data(content)
    
    if not data:
        print("❌ No generator data found in log file")
        return
    
    print(f"Found {len(data['c'])} composite generators:")
    for freq in sorted(data['c'].keys(), key=int):
        print(f"  • {freq}Hz: {len(data['c'][freq]['t'])} data points")
    print(f"Found {len(data['s'])} tone generators:")
    for freq in sorted(data['s'].keys(), key=int):
        print(f"  • {freq}Hz: {len(data['s'][freq]['t'])} data points")
    
    # Find global time range
    all_times = []
    for values in data['c'].values():
        all_times.extend(values['t'])
    for values in data['s'].values():
        all_times.extend(values['t'])

    if not all_times:
        print("❌ No time data found")
        return

    t_min, t_max = min(all_times), max(all_times)
    step = max((t_max - t_min) / 1000, 0.000001)
    
    print(f"\nTime range: {t_min:.6f} to {t_max:.6f}\n")
    
    def plot_range(x_range):
        x_min, x_max = x_range
        plt.figure(figsize=(14, 6))
        
        for freq, values in sorted(data['c'].items(), key=lambda x: int(x[0])):
            mask = [(t >= x_min and t <= x_max) for t in values['t']]
            t_filtered = [t for t, m in zip(values['t'], mask) if m]
            ae_filtered = [ae for ae, m in zip(values['ae'], mask) if m]
            e_filtered = [e for e, m in zip(values['e'], mask) if m]
            
            if t_filtered:
                plt.plot(t_filtered, ae_filtered, marker='o', markersize=4, 
                        label=f'Composite {freq}Hz', alpha=0.7, linewidth=1.5)
                plt.plot(t_filtered, e_filtered, marker='o', markersize=4, 
                        label=f'Composite Envelope {freq}Hz', alpha=0.7, linewidth=1.5)

        if not composite:
            for freq, values in sorted(data['s'].items(), key=lambda x: int(x[0])):
                mask = [(t >= x_min and t <= x_max) for t in values['t']]
                t_filtered = [t for t, m in zip(values['t'], mask) if m]
                ae_filtered = [ae for ae, m in zip(values['ae'], mask) if m]
                e_filtered = [e for e, m in zip(values['e'], mask) if m]
                
                if t_filtered:
                    plt.plot(t_filtered, ae_filtered, marker='o', markersize=4, 
                            label=f'Tone {freq}Hz', alpha=0.7, linewidth=1.5)
                    plt.plot(t_filtered, e_filtered, marker='o', markersize=4, 
                            label=f'Tone Envelope {freq}Hz', alpha=0.7, linewidth=1.5)

        plt.xlabel('Time (t)', fontsize=12)
        plt.ylabel('Amplitude (a)', fontsize=12)
        plt.title('Generator Output Over Time', fontsize=14, fontweight='bold')
        plt.legend(fontsize=10)
        plt.grid(True, alpha=0.3)
        plt.xlim(x_min, x_max)
        plt.tight_layout()
        plt.show()
    
    interact(plot_range,
             x_range=FloatRangeSlider(
                 value=[t_min, t_max],
                 min=t_min,
                 max=t_max,
                 step=step,
                 description='Time Range:',
                 continuous_update=False,
                 readout_format='.6f',
                 style={'description_width': 'initial'},
                 layout={'width': '90%'}
             ))

In [11]:
plot_log("../rustic/app.log", composite=True)

Found 1 composite generators:
  • 440Hz: 44100 data points
Found 6 tone generators:
  • 123Hz: 44100 data points
  • 150Hz: 44100 data points
  • 180Hz: 44100 data points
  • 219Hz: 44100 data points
  • 240Hz: 44100 data points
  • 261Hz: 44100 data points

Time range: 0.000023 to 0.999481



interactive(children=(FloatRangeSlider(value=(2.2675737e-05, 0.9994809), continuous_update=False, description=…